In [1]:
os.chdir('.')

In [2]:
os.getcwd()

'/Users/jamieelliott/Documents/Git/nature_engagement_phase2/point_estimate_model'

In [4]:

from model_config import *
from model_packages import *
from model_utils import *
import re

import test_sites_add_input_data
import add_counter_location_info

/Users/jamieelliott/.pyenv/versions/3.10.9/envs/req_old/bin/python
Confirm Pycaret version is ?
Pycaret Version:  3.0.2


In [ ]:
# Running each script  to generate static features data set for test sites
test_sites_add_input_data.counter_location_process_test_sites_data('./scripts/test_sites_config.json')
test_sites_add_input_data.census_process_test_sites_data()
test_sites_add_input_data.gree_blue_infrastructure_process_test_sites_data()
test_sites_add_input_data.land_classification_process_test_sites_data()
test_sites_add_input_data.land_habitat_process_test_sites_data()
test_sites_add_input_data.dog_process_test_sites_data()
test_sites_add_input_data.pois_process_test_sites_data()

# access config file to loop through providers and datasets
config= add_counter_location_info.get_config_file_paths('./scripts/test_sites_config.json')


# process strava data for each provider
for provider, info in config.items():
    print(f'>>>>>>>>>>>>>{provider, info["strava_path"]}')
    test_sites_add_input_data.strava_process_test_sites_data(provider, info['strava_path'])

# combine counts data for all providers
dfs=[]
for provider, info in config.items():
    df= pd.read_pickle(data_folder+f'test_sites/strava_data_{provider}.pkl')
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df['Date']=combined_df['Date'].astype(str)


# merge with weather features
test_sites_add_input_data.weather_process_test_sites_data()
weather= pd.read_pickle(data_folder+'test_sites_weather_data.pkl')
weather['Date']=weather['Date'].astype(str)
dynamic= weather.merge(combined_df, on= ['Date', 'site'], how = 'right')

# merge with static features
static= pd.read_pickle(data_folder+'test_sites_static_data.pkl')
static_and_dynamic_features= dynamic[['site', 'Date', 'tavg', 'total_trip_count']].merge(static, left_on= 'site', right_on='counter', how= 'inner')

# save data of static and dynamic features for modelling
static_and_dynamic_features.to_pickle(data_folder+f'test_sites_static_and_dynamic_features_{bufr_zones_mrts}.pkl')